# 📊 Evaluation Script

Let's start by reading the data

In [10]:
# DO NOT CHANGE
import cv2
import numpy as np
import glob
import time

## 1. Fill x_test and y_test:
x_test = []
y_test = []
fonts = [ 'IBM Plex Sans Arabic', 'Lemonada', 'Marhey', 'Scheherazade New']

for font in fonts:
    # i = 0 # I was just testing before the submission ..
    for filename in sorted(glob.glob(f'content/test/{font}/*.jpeg')):
        img = cv2.imread(filename, cv2.COLOR_BGR2RGB)
        x_test.append(img)
        y_test.append(fonts.index(font))
        # i = i + 1
        # if (i > 20): break

# 2. Convert them to Numpy arrays:
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print("shape of x_data:", x_test.shape, "shape of y_data:", y_test.shape)
assert set(y_test) == {0, 1, 2, 3}

shape of x_data: (0,) shape of y_data: (0,)


AssertionError: 

Here you define your predict function following this specific interface:

In [8]:
'''
This function takes an RGB image of dimensions (1181, 1181, 3) from the test set and returns integer prediction ∈ {0,1,2,3}
'''
import cv2
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
import os
import numpy as np
from xgboost import XGBClassifier
from cv2.xfeatures2d import SIFT_create as sift_create
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# utility functions
def image_preprocess(img):
    #img = cv2.resize(img, (512, 512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.GaussianBlur(img, (3, 3), 0.1)
    img = cv2.medianBlur(img, 3)  # Kernel size can be adjusted as needed
    histogram = cv2.calcHist([img], [0], None, [256], [0, 256])
    bg_color = np.argmax(histogram)

    thresh = 0
    if bg_color < 50:  # If the background is brighter than a threshold, invert
        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    else:
        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
    return thresh

def apply_pca(descriptors, n_components=64):
    pca = PCA(n_components=n_components)
    descriptors_reduced = pca.fit_transform(descriptors)
    return descriptors_reduced

def create_image_histogram(img, kmeans):
  descriptors = extract_features(img)[1]
  return create_histogram(descriptors , kmeans)

def extract_features(img):
  surf = sift_create()
  keypoints, descriptors = surf.detectAndCompute(img, None)
  return keypoints, descriptors

def create_histogram(features , kmeans , kmeans_sets = 360):
  histogram = np.zeros(kmeans_sets)
  if (features is None or len(features) == 0):
    return histogram
  # descriptors_reduced = apply_pca(features, n_components=64)  # Apply PCA
  prediction = kmeans.predict(features)
  for p in prediction:
    histogram[p] += 1
  return histogram

def classify_image(img, kmeans, clf):
  img = image_preprocess(img)
  histogram = create_image_histogram(img, kmeans)
  predictions = clf.predict([histogram])
  return predictions[0]

classifier_path = "models/classifier_ada.pkl"
kmeans_path     = "models/kmeans_model.pkl"
kmeans = None
clf = None

print("Loading files ..")
print("Preparing Kmeans (Bag of words)")
with open(kmeans_path, 'rb') as f:
    kmeans = pickle.load(f)
if kmeans is not None:
    print("Loaded")
else:
    print("Failed to Load")

with open(classifier_path, 'rb') as f:
    clf = pickle.load(f)
if clf is not None:
    print("Loaded")
else:
    print("Failed to Load")

if kmeans is None or clf is None:
    print("Error loading files")
    
def make_prediction(x):
    return classify_image(x , kmeans , clf)

# Fill your team number here
TEAM_NUM = "14"

Loading files ..
Preparing Kmeans (Bag of words)
Loaded
Loaded


Now let's compute the accuracy of the model:

In [9]:
# DO NOT CHANGE
y_pred = []

start_time = time.time()
for x in x_test:
    assert x.shape == (1181, 1181, 3)
    ŷ = make_prediction(x)
    y_pred.append(ŷ)
end_time = time.time()

y_pred = np.asarray(y_pred)
accuracy = np.mean(y_pred == y_test)
total_time = end_time - start_time
print(f"Team {TEAM_NUM} got accuracy: {accuracy:.2%}")
print(f"Team {TEAM_NUM} got runtime: {total_time:.2%}")

Team 14 got accuracy: 100.00%
Team 14 got runtime: 1737.98%
